# Scraping the FOIs of Coventry City Council

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random

In [2]:

pages = ['A','B','C']

# we'll need this for assembling the full url below
url_base = base = 'https://www.coventry.gov.uk'

# we'll need somewhere to collect all the links
full_list_of_links = []

# assign your user agent to a variable
s_headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'} 

for page in pages:

  # "verify=False" this seemed to return a problem
  # you need to supply the headers in the argument, the code expects: headers = some_variable
  # we don't need to make this 'page' which we're already using for the loop ('for page in....')
  req = requests.get(f'https://www.coventry.gov.uk/directory/45/a_to_z/{page}', headers = s_headers)
  
  # now we make the soup and parse it
  soup = bs(req.content, 'html.parser')
  for infoPage in soup.findAll('ul', class_='item-list'):
      for x in infoPage.findAll('a'):

        # this is how you will get the url without the rest of the text etc.
        link_ending = x.attrs['href']

        # stick the beginning of the url to what you've just collected (you could do this with an f-string too as you did)
        full_link = base + link_ending
        
        # you could print each link, but let's collect them instead
        # print(full_link)
        full_list_of_links.append(full_link)

  # you need the sleep METHOD from the time MODULE (they have to be imported above) - the argument is seconds so 120 is too long
  # if you want to pick a number at random (4, 11) then you need the RANDOM module as well (import random)
  time.sleep(4)
  print('one loop') # just to keep track
  # time.sleep(random.randint(4,11))

one loop
one loop
one loop


In [3]:
print(len(full_list_of_links))

1509


In [ ]:
for x in full_list_of_links:
  print(x)

## build a df from these links - or one of them to start with

In [6]:
# assign a single url from the list as the url to scrape next - to test it works with a single page

url = full_list_of_links[4]
print(url)

req = requests.get(url, headers = s_headers)

print(req.status_code)

foi_soup = bs(req.content)

200


In [75]:
# parse the soup, just printing what's needed

for x in foi_soup.find_all('h1'):
  title = x.text
  print(title)
  print('****')

for x in foi_soup.find_all('div', class_ = 'record_item'):
    for y in x.find_all('h2'):
      print(y.text)
    for z in x.find_all('p'):
      print(z.text)
    print('---')

# the 'h2' part doesn't change so it will be enough to collect all the Ps into a list and append

FOI and EIR requests - A45 North Bound outside lane road maintenance
****
Request reference
REQ00978
---
Request date
17 February 2016
---
Response date
15 March 2016
---
Closed date
15 March 2016
---
Category
Parking, travel and streets
---
Outcome
Some information provided
---
Response
http://www.coventry.gov.uk/downloads/file/20276/req00978
---


In [76]:
# ultimately we'll need to build a list of lists, with each smaller list gathering from one url
total_list = []

# build a list from this single url
some_list = []
for x in foi_soup.find_all('h1'):
  title = x.text
  some_list.append(title)

for x in foi_soup.find_all('div', class_ = 'record_item'):
    for z in x.find_all('p'):
      some_list.append(z.text)

total_list.append(some_list)

In [77]:
# now it only has one list in it, but when we get all the urls it will be a list of lists
total_list

[['FOI and EIR requests - A45 North Bound outside lane road maintenance',
  'REQ00978',
  '17 February 2016',
  '15 March 2016',
  '15 March 2016',
  'Parking, travel and streets',
  'Some information provided',
  'http://www.coventry.gov.uk/downloads/file/20276/req00978']]

In [78]:
import pandas as pd

# define the column names
my_columns = ['subject', 'ref', 'date_req', 'date_resp', 'closed_date', 'cat', 'outcome', 'link_pdf_page']

# build the df from the list of lists
dfl = pd.DataFrame(total_list, columns = my_columns)

In [79]:
dfl

,subject,ref,date_req,date_resp,closed_date,cat,outcome,link_pdf_page
0,FOI and EIR requests - A45 North Bound outside...,REQ00978,17 February 2016,15 March 2016,15 March 2016,"Parking, travel and streets",Some information provided,http://www.coventry.gov.uk/downloads/file/2027...
